<a href="https://colab.research.google.com/github/GirishaSingh/Fraud-Detection-With-Generative-AI/blob/main/GAN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#GAN Model

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

In [4]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [5]:
def train_gan(generator, discriminator, data_loader, num_epochs=100):
    criterion = nn.BCELoss()
    optimizer_g = optim.Adam(generator.parameters(), lr=0.0002)
    optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002)

    for epoch in range(num_epochs):
        for real_data in data_loader:
            # Train discriminator
            optimizer_d.zero_grad()
            real_labels = torch.ones(real_data.size(0), 1)
            fake_data = generator(torch.randn(real_data.size(0), generator.input_dim))
            fake_labels = torch.zeros(real_data.size(0), 1)
            loss_d_real = criterion(discriminator(real_data), real_labels)
            loss_d_fake = criterion(discriminator(fake_data.detach()), fake_labels)
            loss_d = (loss_d_real + loss_d_fake) / 2
            loss_d.backward()
            optimizer_d.step()

            # Train generator
            optimizer_g.zero_grad()
            loss_g = criterion(discriminator(fake_data), real_labels)
            loss_g.backward()
            optimizer_g.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}/{num_epochs} - Loss D: {loss_d.item()}, Loss G: {loss_g.item()}')

In [6]:
generator = Generator(input_dim=100, output_dim=28*28)
discriminator = Discriminator(input_dim=28*28)

In [7]:
# Now we can use the collected data for training
# train_gan(generator, discriminator, data_loader)